# Lab 12-5 sequence to sequence (Keras + eager version)

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

In [1]:
pip install tensorflow==1.15

  Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os
import pandas as pd
import re
#요약
from gensim.summarization.summarizer import summarize




1.15.0


In [20]:
import json
with open('movies.json', 'rt',encoding='UTF8') as datafile:
    data = json.load(datafile)
df = pd.DataFrame(data)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa9 in position 1184452: invalid start byte

In [5]:
df

NameError: name 'df' is not defined

# 전처리

In [28]:
#df = movie_dataframe

df.movie_name = df.movie_name.apply(lambda x: x.replace('혻',''))

df.movie_summary = df.movie_summary.apply(lambda x: str(x).lower())
df.movie_summary = df.movie_summary.apply(lambda x: re.sub("[^A-Za-z. ]", '', x))
df.movie_summary = df.movie_summary.apply(lambda x: x.replace('\\n',''))

#df.movie_synopsis = df.movie_synopsis.apply(lambda x : str(x).lower())
#df.movie_synopsis = df.movie_synopsis.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))

df.movie_tagline = df.movie_tagline.apply(lambda x: str(x).lower())
df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('\\n','').replace('\\xa0',''))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub(' {2,}', '', x))

#태그라인 없는 데이터 row 삭제
index = df[df['movie_tagline']==''].index
df.drop(index, inplace = True)

#summary 없는 데이터 row 삭제
summary_index = df[df['movie_summary']==''].index
df.drop(summary_index, inplace = True)

index = df[df['movie_summary'].str.contains('  ')].index
df.drop(index , inplace = True)




In [29]:
df= df.reset_index()

In [30]:
df

,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
0,1000,"['Drama', 'Fantasy', 'Romance', 'Thriller']",Beauty and the Beast,www.imdb.com/title/tt2316801/mediaviewer/rm240...,an unexpected romance blooms after the the you...,A widowed merchant Andr Dussollier is forced t...,the legend is reborn...
1,1002,"['Comedy', 'Drama', 'Romance']",Bob & Carol & Ted & Alice,www.imdb.com/title/tt0064100/mediaviewer/rm269...,a lighthearted picture of middleaged love and ...,n n,consider the possibilities
2,1003,"['Animation', 'Action', 'Adventure', 'Crime', ...",Batman: Mask of the Phantasm,www.imdb.com/title/tt0106364/mediaviewer/rm168...,batman is wrongly implicated in a series of mu...,A meeting between crime lord Chuckie Sol Dick ...,the dark knight fights to save gotham city fro...
3,1005,"['Drama', 'Mystery', 'Thriller']",Stay,www.imdb.com/title/tt0371257/mediaviewer/rm128...,this movie focuses on the attempts of a psychi...,The film opens with a car crash on Brooklyn Br...,between the worlds of the living and the dead ...
4,1006,['Comedy'],Down Periscope,www.imdb.com/title/tt0116130/mediaviewer/rm272...,lt. cmdr. tom dodge is assigned as captain to ...,Lieutenant Commander Thomas Dodge Kelsey Gramm...,a rusty sub. a rebel commander. a renegade cre...
5,1009,"['Comedy', 'Drama', 'Family', 'Music', 'Romance']",Hannah Montana: The Movie,www.imdb.com/title/tt1114677/mediaviewer/rm229...,as hannah montanas popularity begins to take o...,Miley Stewart has always had the best of both ...,she has the best of both worlds... now she has...
6,101,"['Comedy', 'Horror']",Shaun of the Dead,www.imdb.com/title/tt0365748/mediaviewer/rm418...,a mans uneventful life is disrupted by the zom...,The film begins in The Winchester a traditiona...,ever felt like you were surrounded by zombies
7,1010,"['Action', 'Crime', 'Thriller']",El Mariachi,www.imdb.com/title/tt0104815/mediaviewer/rm280...,a traveling mariachi is mistaken for a murdero...,In a small Mexican town a ruthless criminal ni...,he didnt come looking for trouble but trouble ...
8,1011,"['Drama', 'Romance']",Damage,www.imdb.com/title/tt0104237/mediaviewer/rm361...,a member of parliament falls passionately in l...,Dr. Stephen Fleming Jeremy Irons a British cab...,with love comes risk. with obsession comes... ...
9,1014,"['Comedy', 'Drama', 'Romance']",Show Me Love,www.imdb.com/title/tt0150662/mediaviewer/rm132...,two teenage girls in smalltown sweden. elin is...,n n,jag ska aldrig mer bli ihop med nn. jag ska bl...


In [31]:

tl = list(df["movie_tagline"])
tl = ["".join([ j for j in i if j != "\n"]).strip() for i in tl]

summ = list(df['movie_summary'])

df.movie_tagline = tl
df.movie_summary = summ

sources = [i.split() for i in df['movie_summary']]
targets = [i.split() for i in df['movie_tagline']]


In [32]:
df.tail()

,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
1905,988,"['Comedy', 'Family', 'Sport']",The Big Green,www.imdb.com/title/tt0112499/mediaviewer/rm267...,a teacher on exchange from england is placed i...,n n,the biggest kick of the year
1906,991,"['Horror', 'Thriller']",Embrace of the Vampire,www.imdb.com/title/tt0109723/mediaviewer/rm618...,an yearold college freshman is seduced by a ha...,From the diary of the vampire Martin Kemp I wa...,between innocence and evil lies the seduction.
1907,992,"['Drama', 'Romance']",Daddy's Little Girls,www.imdb.com/title/tt0778661/mediaviewer/rm197...,a mechanic falls in love with his attorney. a ...,n n,she needs a good man. he wants a smart woman. ...
1908,996,"['Comedy', 'Crime', 'Drama', 'Sport']",The Longest Yard,www.imdb.com/title/tt0071771/mediaviewer/rm576...,a sadistic warden asks a former pro quarterbac...,n n,the audiences dont just see itthey talk to it ...
1909,997,"['Horror', 'Thriller']",A Haunting at Silver Falls,www.imdb.com/title/tt1813609/mediaviewer/rm339...,a small town is haunted by the twin daughters ...,n n,twin girls haunt a new girl in town to steer h...


# 단어 나누기

In [33]:

# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}



In [34]:
# vocabulary for targets
t_vocab = list(set(sum(targets, [])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}




In [35]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    
    if mode == 'source':
        # preprocessing for source (encoder)
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        
        #s_input = [[x for x in each_input if x] for each_input in s_input]
        #pprint(s_input)
        s_len = list(map(lambda sentence : len(sentence), s_input))
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input
    

    elif mode == 'target':
        # preprocessing for target (decoder)
        # input
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        
        #t_input = [[x for x in each_input if x] for each_input in t_input]
        #pprint(t_input)
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
     
        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
        
        return t_len, t_input, t_output

In [100]:
sum_ = 0
for i in df.movie_summary:
    length = len(i.split())
    sum_ += length
average_no = sum_ / len(df) #summary당 평균 182.84 단어
print(average_no)
# preprocessing for source
#단어 수는 120개로 제한
s_max_len = int(average_no)-100
print(s_max_len)
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
#print(s_len, s_input)

160.10523560209424
60


In [101]:
sum_ = 0
for i in df.movie_tagline:
    length = len(i.split())
    sum_ += length
average_no = sum_ / len(df) #각 tagline별 단어 수는 9.6개
# preprocessing for target
#단어 수 10개로 제한
t_max_len = int(average_no)
print(t_max_len)
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
#print(t_len, t_input, t_output)

9


# hyper-param

In [121]:
# hyper-parameters
epochs = 100
batch_size = 30
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units =64

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [122]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [123]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [124]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
                
    def call(self, x, hidden, enc_output):
        
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [125]:
print(source2idx)

{'<pad>': 0, '.': 1, '..': 2, '...': 3, '.next': 4, '.sonia': 5, 'a': 6, 'a.': 7, 'a.a.': 8, 'a.c.o.d.': 9, 'a.d.': 10, 'a.k.a.': 11, 'a.s.a.p': 12, 'aaron': 13, 'aaronnow': 14, 'aarons': 15, 'aaryan': 16, 'abadoned': 17, 'abandon': 18, 'abandon..': 19, 'abandonded': 20, 'abandoned': 21, 'abandonment': 22, 'abandons': 23, 'abarra': 24, 'abarras': 25, 'abb': 26, 'abbe': 27, 'abbey': 28, 'abbey.': 29, 'abbot': 30, 'abbott': 31, 'abby': 32, 'abdel': 33, 'abdels': 34, 'abdomen.': 35, 'abduct': 36, 'abducted': 37, 'abducted....': 38, 'abducting': 39, 'abduction': 40, 'abductions.': 41, 'abductor.': 42, 'abductors': 43, 'abductors.': 44, 'abducts': 45, 'abe': 46, 'abel': 47, 'abels': 48, 'abercrombie': 49, 'abhors': 50, 'abides': 51, 'abiding': 52, 'abigail': 53, 'abigail.': 54, 'abilene': 55, 'abilities': 56, 'abilities.': 57, 'ability': 58, 'ability.': 59, 'able': 60, 'ablixa': 61, 'ablixa.': 62, 'abnormal': 63, 'aboard': 64, 'abode': 65, 'abolishment': 66, 'abolitionist': 67, 'abominable'

In [126]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    
#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))
    
    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)
print(checkpoint_prefix)
# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir=checkpoint_dir)
print(summary_writer)

./data_out/training_checkpoints\ckpt


In [127]:
EPOCHS = 100

def optimized(g, v):
    return optimizer.apply_gradients(zip(g, v))

def minimized(v):
    return optimizer.minimize(v)

for epoch in range(EPOCHS):
    
    hidden = encoder.initialize_hidden_state()
    #print(hidden.shape)
    total_loss = 0
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        #print(s_input, s_input.shape)
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(s_input, hidden)

            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #Teacher Forcing: feeding the target as the next input
            for t in range(1, t_input.shape[1]):
                
                predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(t_input[:, t], predictions)
            
                dec_input = tf.expand_dims(t_input[:, t], 1) #using teacher forcing
                
        batch_loss = (loss / int(t_input.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        #print(variables)
        gradient = tape.gradient(loss, variables)
        #optimized(gradient, variables)
        #print(gradient, variables)
        optimizer.apply_gradients(zip(gradient, variables))
        #print(optimizer.apply_gradients(zip(gradient, variables)))


    if epoch % 10 == 0:
        #save model every 10 epoch
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                            total_loss / n_batch,
                                            batch_loss.numpy()))
        #minimized(variables)
        checkpoint.save(file_prefix = checkpoint_prefix)


InvalidArgumentError: Incompatible shapes: [18,64] vs. [30,64] [Op:AddV2] name: encoder_11/gru_22/add/

In [115]:
# #최근 학습된 모델을 불러오고 이어서 EPOCHS로 돌아감
# saver = tf.train.Saver()
# with tf.Session() as sess:
    
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [116]:
#sentence = 'I feel hungry'
sentence = df.movie_summary[7]
print(sentence)
print(df.movie_name[7])
print(df.movie_tagline[7])
#sentence = 'desperado'
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

a traveling mariachi is mistaken for a murderous criminal and must hide from a gang bent on killing him. el mariachi just wants to play his guitar and carry on the family tradition. unfortunately the town he tries to find work in has another visitor...a killer who carries his guns in a guitar case. the drug lord and his henchmen mistake el mariachi for the killer azul and chase him around town trying to kill him and get his guitar case. el mariachi just simply wants to play his guitar no harm done no questions asked. unfortunately he makes the mistake of attempting to find work in a local town just when an assassin who keeps his guns in a guitar case hits town. el mariachi is mistaken for the assassin. not good. american crime lord moco has set up a lucrative business in mexico. he tries to rub out his imprisoned employee azul. however azul overcomes the hitmen and escapes determined to get his revenge. meanwhile a wandering mariachi comes to the same town looking for work. as azuls tr

In [117]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
        
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
        
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
   
    for t in range(max_length_targ):
        

        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '
        
        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)    
            
    return result, sentence
    
result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

print(sentence)
print(result)

a traveling mariachi is mistaken for a murderous criminal and must hide from a gang bent on killing him. el mariachi just wants to play his guitar and carry on the family tradition. unfortunately the town he tries to find work in has another visitor...a killer who carries his guns in a guitar case. the drug lord and his henchmen mistake el mariachi for the killer azul and chase him around town trying to kill him and get his guitar case. el mariachi just simply wants to play his guitar no harm done no questions asked. unfortunately he makes the mistake of attempting to find work in a local town just when an assassin who keeps his guns in a guitar case hits town. el mariachi is mistaken for the assassin. not good. american crime lord moco has set up a lucrative business in mexico. he tries to rub out his imprisoned employee azul. however azul overcomes the hitmen and escapes determined to get his revenge. meanwhile a wandering mariachi comes to the same town looking for work. as azuls tr

In [118]:
#len(df.movie_summary[39].split('. '))

In [119]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    inputs = []
    x = sentence.split(' ')
    #keyerror 처리
    for i in x:
        if not i in inp_lang.keys():
            pass
        else:
            inputs.append(inp_lang[i])
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    print(inputs)
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    idx = 0
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    #print(dec_input)
    xList = []
    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
        #print("*"*100)
        #print(predictions)
        #print("*"*100)
        #print(tf.argmax(predictions[0]))
        print(predictions)
        x = tf.nn.top_k(predictions[0], k=3, sorted=True, name=None)
        for i in range(3):
            predicted_id = x[1][i].numpy()
            #print(predicted_id)

            result += idx2target[predicted_id] + ' '
            #print("result:", result)
            #print(idx2target[predicted_id])
            #if idx2target.get(predicted_id) == '<eos>':
            #             return result, sentence

          #result=""

        # the predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)  
        xList.append([result])
        result = ''
        
    return xList

# result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
tList=prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
# print(sentence)
from pprint import pprint
pprint(tList)

tf.Tensor(
[[ 8271  6607 24993 10235 11541 20761 15519     6 26706 15187  4834 24993
  24716 21457 25168 14621  9520 27343  1391  1750 25215 12925 11541 10789
   3822  9184 27255 26866 24716 15187 12925 16038  9520 11508   905  9225
  11515     6 11955 15040  8401 14621  9520 11508 14358 24993 22368  9271
    905  1415 19728 13996 22929  5414 24716 16053 27141 17113  2104  7617]], shape=(1, 60), dtype=int32)
tf.Tensor([[-3.8906546 -3.685345   4.2918043 ... -3.5449178 -3.9228003 -3.8278742]], shape=(1, 4643), dtype=float32)
tf.Tensor([[-4.240696  -4.0903225  4.8475738 ... -3.8738623 -4.2728233 -4.18891  ]], shape=(1, 4643), dtype=float32)
tf.Tensor([[-4.362361  -4.2544527  5.135034  ... -4.00306   -4.3977156 -4.331863 ]], shape=(1, 4643), dtype=float32)
tf.Tensor([[-4.404798  -4.3215094  5.269692  ... -4.0539627 -4.442353  -4.3928866]], shape=(1, 4643), dtype=float32)
tf.Tensor([[-4.4200187 -4.348805   5.3285694 ... -4.0740237 -4.458881  -4.4186745]], shape=(1, 4643), dtype=float32)
tf.